<a href="https://colab.research.google.com/github/Chanchalg05/Deep_learning/blob/main/CNN_model_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 44.7 MB/s 


In [3]:
import tensorflow as tf


In [4]:
from tensorflow import keras
import numpy as np

In [5]:
fashion_mnist= tf.keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

<ipython-input-14-d9799366288c>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [15]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.9066666960716248

Best val_accuracy So Far: 0.9160000085830688
Total elapsed time: 00h 02m 45s


In [18]:
model=tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        27712     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 96)                3539040   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,568,202
Trainable params: 3,568,202
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1112 - accuracy: 0.9581 - val_loss: 0.2594 - val_accuracy: 0.9122
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0748 - accuracy: 0.9722 - val_loss: 0.3048 - val_accuracy: 0.9130
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0491 - accuracy: 0.9816 - val_loss: 0.3616 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0378 - accuracy: 0.9861 - val_loss: 0.3697 - val_accuracy: 0.9117
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0292 - accuracy: 0.9894 - val_loss: 0.4150 - val_accuracy: 0.9125
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0225 - accuracy: 0.9924 - val_loss: 0.4944 - val_accuracy: 0.9170
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0176 - accuracy: 0.9940 - val_loss: 0.5457 - val_accuracy